# 15.066 Summer 2025
## System Optimization and Analysis for Operations
### final Project - Group 5A
---
Grayson Adams, Shane Pornprinya, Becca Sholler

---

## Load Packages

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from project_data import *
# %whos

In [189]:
import numpy as np
import os, pathlib
import pandas as pd
FILE_DIR = os.getcwd()

# Fuel Types
FUEL = ['coal', 'NGCC', 'nuclear', 'hydro', 'wind', 'solar', 'ev']

# Carbon Emission in gCO2e per kWh for each fuel type
E = np.array([820, 490, 12, 24, 11.5, 44.5, 0,])

# LCOE in $/kWh for each fuel type from AEO2023
C = np.array([89.33, 42.72, 71.00, 57.12, 31.07, 23.22, 36.27])/1e3

# Number of fuel types
NUMFT = len(E)

# State level data on population, energy demand, current co2 emissions from electricity
FNAME = os.path.join(FILE_DIR, "StateData.xlsx")
STATES = pd.read_excel(FNAME, sheet_name="StateData", index_col=0, header=0)

# EV Capacity (avg) in kWh
EVC_MAX = 80

# Daily EV energy usage in kWh
EVC_MIN = 5*11106/3.5/365 # safety factor * avg annual miles / avg miles/kWh / days per year 

# Population (of the county)
# FNAME = os.path.join(FILE_DIR, "EVs.csv") # TODO fix filename
# EVS = pd.read_csv(FNAME, dtype={'county': str, 'population': float, 'numEV': float})
# EVS['EVB_maxcapacity'] = EVS['numEV'] * EVC_MAX
# EVS['EVB_mincapacity'] = EVS['numEV'] * EVC_MIN

### Approach 1
MAXGEN = [.85, .87, .9, .56, .4, .29, 0] #TODO, Using capacity factor, but does that just represent uptime and we need actual capacity
MINGEN = [0, 0, .25, 0, 0, 0, 0,] #TODO, should be expressed as a minimum kWh number??
# TODO how to account for these going up with EV bank capacity? constrain dec vars for these three + ev bank to their sum, also individual constrs

### Approach 3
HRLYDEMAND = np.array([1, 1, 1, 1, 1, 1,
                       1, 1, 1, 1, 1, 1,
                       1, 1, 1, 1, 1, 1,
                       1, 1, 1, 1, 1, 1,]) # TODO Real data, maybe normalized then multiply by DEMANDPC['<state>']
HRLYMAXGEN = pd.DataFrame() # TODO 7x24 array, 1 and 2 are inf, others need research. Express as pct of demand so it can scale to county
HRLYPRICE = pd.DataFrame() # TODO 7x24 array, most are constant. Units $/kWh

In [190]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 1)
env.start()

# Initialize the model
m = gp.Model(env=env)

Set parameter Username
Set parameter LicenseID to value 2676206
Academic license - for non-commercial use only - expires 2026-06-08


---
## Optimization Case Details

In [191]:
state = 'WA'
futureDemandGrowth = 1.5 # Future electricity demand as pct of current
TotalDemand = STATES['Demand (MWh)'][state]*1e3*futureDemandGrowth # convert to kwh
futureCo2Improve = 0.99 # Future emissions target as pct of current
LimitCO2 = STATES['CO2 Emissions (million metric tons)'][state]*1e12*futureCo2Improve #convert to gCO2e

---
## Decision Variables

In [192]:
### Decision Variables ###

## Continuous
# A vector of qty of each fuel type
F = m.addMVar(NUMFT, vtype=GRB.CONTINUOUS, name='Fuel') # For single day

## Binary
# Whether or not to use each fuel type (due to minimum generation constraints)
X = m.addMVar(NUMFT, vtype=GRB.BINARY, lb=0) 

---
## Objective Function

In [193]:
# Objective function: minimize cost
m.setObjective(gp.quicksum(F[i]*C[i] for i in range(NUMFT)),GRB.MINIMIZE)

---
## Constraints

In [194]:
### Constraints ###
# Total production must meet demand
Const1 = m.addConstr(gp.quicksum(F[i] for i in range(NUMFT))>=TotalDemand)

# Total CO2 emissions must be less than limit
Const2 = m.addConstr(gp.quicksum(F[i]*E[i] for i in range(NUMFT))<=LimitCO2)

# Generation is capacity limited
Const3 = m.addConstrs(F[i] <= MAXGEN[i]*TotalDemand*X[i] for i in range(NUMFT))

# Min generation requirements
Const4 = m.addConstrs(F[i] >= MINGEN[i]*TotalDemand*X[i] for i in range(NUMFT))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


---
## Solve

In [195]:
# Update and write the model
m.update() # Update model parameters
m.write("15066_final_project_5A.lp") # Write model to file
m.optimize() # Solve

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 14 columns and 34 nonzeros
Model fingerprint: 0x22cf6201
Variable types: 7 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+11]
  Objective range  [2e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+11, 1e+13]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 12 rows and 8 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 14 nonzeros
Variable types: 5 continuous, 1 integer (1 binary)
Found heuristic solution: objective 4.676775e+09

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [196]:
# Print optimal objective function value
initObjVal = m.getAttr("ObjVal")
print("\nObjective value (total generation cost $):", "%.0f" % (initObjVal))
optCost = initObjVal*100/TotalDemand
print("\nOptimized avg retail price (cents/kWh):", "%.2f" % (optCost))
currCost = STATES['Average retail price (cents/kWh)'][state]
print("\nCurrent per capita generation cost (cents/kWh):", "%.2f" % (currCost))



Objective value (total generation cost $): 4676774800

Optimized avg retail price (cents/kWh): 3.48

Current per capita generation cost (cents/kWh): 10.26


In [197]:
# Print the optimal decsion variable solutions
initSoln = pd.DataFrame((F.getAttr('x')/1e3).round(0), index=FUEL, columns=['MWh Generated'])
display(initSoln)

,MWh Generated
coal,0.0
NGCC,19152713.0
nuclear,0.0
hydro,22489260.0
wind,53731578.0
solar,38955394.0
ev,0.0


---
## Review Constraints

In [198]:
# Calculate used budget
# leftoverBudget = m.getConstrByName('Const5').Slack

---
## Senstivity Analysis

In [199]:
# newTotalBudget = TotalBudget*1.1
# # Hint: You can change the value of the total budget with this command, by changing the right
# # hand side of Constraint 5, be sure constraint 5 is named 'Const5'
# m.getConstrByName('Const5').setAttr('RHS', newTotalBudget)

# # Re-solve
# m.update()
# m.optimize()

# # Print optimal objective function value
# newObjVal = m.getAttr("ObjVal")
# print(f"New Co2 Emissions Objective value: ","%.0f" %  newObjVal)
# print(f"Pct Change in Objective value: ","%.1f" % ((newObjVal-initObjVal)/initObjVal*100))

# newSoln = pd.DataFrame(SCM.getAttr('x'), index=ODM_names, columns=transport_names)
# print("Change from Initial Solution")
# display(newSoln-initSoln)

# # sensitivity table
# budget_sensitivity = [1.0, 1.02, 1.04, 1.06, 1.1, 1.12, 1.14,] # FILL-IN

# # We will create a vector of emissions by looping through budget_sensitivity
# emissions = []
# for sensitivity in budget_sensitivity:
#     newTotalBudget = TotalBudget*sensitivity
#     m.getConstrByName('Const5').setAttr('RHS', newTotalBudget)

#     # Re-solve
#     m.update()
#     m.optimize()

#     # Print optimal objective function value
#     emissions.append(m.getAttr("ObjVal"))
    
# print(emissions)

In [200]:
# # Graph results
# plt.figure(figsize=(10, 6))
# plt.plot([((x - 1) * 100) for x in budget_sensitivity], emissions, marker='o', linestyle='-', color='b')
# plt.title('CO2 Emissions vs Budget Increase', fontsize=14)
# plt.xlabel('% Budget Increase', fontsize=12)
# plt.ylabel('CO2 Emissions', fontsize=12)
# plt.grid(True)
# plt.show()